# Deep Learning Neural Network on Animal Shelter data

In [21]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import theano

import matplotlib.pyplot as plt
import os


In [2]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [4]:
# Set up my data directories from different machines

mac_data_dir = '/Users/christopherallison/Documents/Coding/Data'
linux_data_dir = '/home/chris/data'
win_data_dir = u'C:\\Users\\Owner\\Documents\\Data'

# Set data directory for example

data_dir = win_data_dir

In [11]:
# Load our prepared dataset and reference data

df = pd.read_csv(os.path.join(data_dir, "prepared_dogs_df.csv"),index_col=0)

In [12]:
df.head()

,OutcomeType,SexuponOutcome,AgeuponOutcome,Color,NameLength,BreedKMeans
2,Adoption,0.0,2,0,6,0
5,Transfer,1.0,0,1,4,4
12,Adoption,0.0,2,2,6,0
20,Transfer,2.0,0,3,6,0
31,Transfer,2.0,0,3,6,3


In [13]:
# Drop uneccesary columns
X = df.drop('OutcomeType', axis=1)
X.dtypes

SexuponOutcome    float64
AgeuponOutcome      int64
Color               int64
NameLength          int64
BreedKMeans         int64
dtype: object

In [14]:
X.head()

,SexuponOutcome,AgeuponOutcome,Color,NameLength,BreedKMeans
2,0.0,2,0,6,0
5,1.0,0,1,4,4
12,0.0,2,2,6,0
20,2.0,0,3,6,0
31,2.0,0,3,6,3


In [15]:
X = X.as_matrix()

In [17]:
outcomes = df.OutcomeType.unique()

In [18]:
from sklearn import preprocessing

# This code takes our text labels and creates an encoder that we use
# To transform them into an array

encoder = preprocessing.LabelEncoder()
encoder.fit(outcomes)

encoded_y = encoder.transform(outcomes)
encoded_y

#We can also inverse_transform them back.
list(encoder.inverse_transform([0, 1, 2, 3]))

#We still need to transform the array into a matrix - this is called one hot encoding. It allows us to track the probability of each possible outcome separately.
# convert integers into dummy variables (one hot encoding)

from keras.utils import np_utils

# We'll practice with the dummy for now, but we'll transform the full labels later.
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

#We'll transform the labels into their array value, though.
df.OutcomeType = encoder.transform(df.OutcomeType)

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [19]:
train_target = np_utils.to_categorical(df['OutcomeType'].values)
train_target

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.]])

In [22]:
model = Sequential()
model.add(Dense(4, input_dim=5, init='normal', activation="relu"))
model.add(Dense(4, init='normal', activation='sigmoid'))

# Compile model
print("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

Compiling model...


In [25]:
hist = model.fit(X, train_target, validation_split=0.2)
print("")
print(hist.history)

Train on 3160 samples, validate on 790 samples
Epoch 1/10
3160/3160 [==============================] - 2s - loss: 0.8744 - acc: 0.6769 - val_loss: 0.8717 - val_acc: 0.6759
Epoch 2/10
3160/3160 [==============================] - 2s - loss: 0.8732 - acc: 0.6769 - val_loss: 0.8703 - val_acc: 0.6759
Epoch 3/10
3160/3160 [==============================] - 2s - loss: 0.8723 - acc: 0.6769 - val_loss: 0.8696 - val_acc: 0.6759
Epoch 4/10
3160/3160 [==============================] - 2s - loss: 0.8705 - acc: 0.6769 - val_loss: 0.8682 - val_acc: 0.6759
Epoch 5/10
3160/3160 [==============================] - 2s - loss: 0.8688 - acc: 0.6769 - val_loss: 0.8668 - val_acc: 0.6759
Epoch 6/10
3160/3160 [==============================] - 2s - loss: 0.8673 - acc: 0.6769 - val_loss: 0.8654 - val_acc: 0.6759
Epoch 7/10
3160/3160 [==============================] - 2s - loss: 0.8654 - acc: 0.6769 - val_loss: 0.8632 - val_acc: 0.6759
Epoch 8/10
3160/3160 [==============================] - 2s - loss: 0.8626 - ac

In [24]:
model.evaluate(X, train_target)

3936/3950 [============================>.] - ETA: 0s

[0.87384914026984684, 0.67670886072931413]

In [26]:
model.predict_proba(X[0:2])

2/2 [==============================] - 0s


array([[ 0.27300504,  0.00991394,  0.25666547,  0.98365349],
       [ 0.29328531,  0.00851487,  0.16190167,  0.98544514]])